In [1]:
def add_time_param(df):
    df['Date'] = pd.DatetimeIndex(df['Date'])
    df['Year'] = [x.year for x in df['Date']]
    df['Month'] = [x.month for x in df['Date']]
    df['Day'] = [x.day for x in df['Date']]
    df['WeekOfYear'] = [x.weekofyear for x in df['Date']]
    return df


In [ ]:
def process_store(store):
    store['CompetitionDistance'].fillna(store['CompetitionDistance'].median(), inplace = True)
    store.fillna(0, inplace = True)
    return store

In [3]:
def get_processed_data_with_store(df):
    # 1. add time param 
    df = add_time_param(df)
    store = process_store(store)
    df_store = pd.merge(df, store, how = 'inner', on = 'Store')
    
    df_store['CompetitionOpen'] = 12 * (df_store.Year - df_store.CompetitionOpenSinceYear) + \
            (df_store.Month - df_store.CompetitionOpenSinceMonth)
    # Promo open time
    df_store['PromoOpen'] = 12 * (df_store.Year - df_store.Promo2SinceYear) + \
            (df_store.WeekOfYear - df_store.Promo2SinceWeek) / 4.0
    
    map_dict = {
        'a': 1,
        'b': 2,
        'c': 3,
        'd': 4,
        '0': 0,
        0: 0,
        1: 1,
        2: 2,
        3: 3,
        4: 4,
        'Jan,Apr,Jul,Oct': 1,
        'Feb,May,Aug,Nov': 2,
        'Mar,Jun,Sept,Dec': 3
    }
    
    df_store.StateHoliday = df_store.StateHoliday.map(map_dict)
    df_store.StoreType = df_store.StoreType.map(map_dict)
    df_store.Assortment = df_store.Assortment.map(map_dict)
    df_store.PromoInterval = df_store.PromoInterval.map(map_dict)
    df_store.PromoOpen = df.PromoOpen.apply(lambda x: x if x > 0 else -1)
    df_store.CompetitionOpen = df.CompetitionOpen.apply(lambda x: x if x > 0 else -1)
    
    return df_store

In [2]:
features = [
    'Store',
    'Open',
    'PromoOpen',
    'DayOfWeek',
    'StateHoliday',
    'StoreType',
    'WeekOfYear',
    'Year',
    'SchoolHoliday',
    'Assortment',
    'Promo',
    'CompetitionDistance']

In [4]:
def rmspe(y_true, y_pred):
    '''
    Compute Root Mean Square Percentage Error between two arrays.
    '''
    loss = np.sqrt(np.mean(np.square(((y_true - y_pred) / y_true)), axis=0))

    return loss

def rmspe_lgb(yhat, y):
    return "rmspe", rmspe(y, yhat), False